In [4]:
alpha_name = "AlphaNetV1_Shift_1_Sequence_20_10d_ret_standscalar"
start_date = '2019-01-01'
end_date = '2021-06-01'
change = 10

In [5]:
import time
from Research.backtest.bt import BTDaily
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
import numpy as np
import pandas as pd
import DataAPI
import os
input_path = r'/home/wuwenjun/factor_lib/ti0/wuwenjun/' + alpha_name
output_path = r'/home/wuwenjun/factor_lib/ti0/wuwenjun/'

alpha_list = []
trading_list = DataAPI.get_trading_days(start_date, end_date)
count = 0
for day in trading_list:
    file = os.path.join(input_path, str(day.year), day.strftime('%Y%m%d') + '.csv')
    try:
        df = pd.read_csv(file, dtype={0: str})
    except FileNotFoundError:
        count += 1
        continue
    flag = count % change
    path_output = os.path.join(output_path, str(alpha_name + "_" + str(flag)), str(day.year), day.strftime('%Y%m%d') + '.csv')
    alpha_list.append(str(alpha_name + "_" + str(flag)))
    if not os.path.exists(path_output):
        os.makedirs(os.path.dirname(path_output), exist_ok=True)
    df.to_csv(path_output, index=False, header=False)
    count += 1

In [6]:
pd.set_option('expand_frame_repr', False)
config_path = r'/home/ShareFolder/lgc/Modules/Research/config/feature_bt_template'
print('Loading the configuration from ' + config_path)
configs = namespace.load_namespace(config_path)
FT = FeatureAnalysis(configs, feature_path=r"/home/wuwenjun/factor_lib")
alpha_list = list(set(alpha_list))
print(alpha_list)
FT.load_feature_from_file(alpha_list, start_date, end_date, universe='Investable',
                                       timedelta=None, transformer=norm.standard_scale)
FT.load_return_data()

writing_path = r'/home/wuwenjun/Desktop/%s_%ddChange.txt' % (alpha_name,change)
f = open (writing_path,'w')
print("\n",file=f)
f.close()
for alpha_name in alpha_list:
    FT.get_intersection_ic(feature_name=alpha_name, truncate_fold=None, method='spearman',
                                     period=('1d', '3d', '5d'))
    ic_flag, trading_direction = FT.test_ic(alpha_name, verbose=False)
    df, df_all = FT.get_ic_summary_by_month(num=6)
    
    if trading_direction == -1:
        negative = True
    else:
        negative = False
    f = open (writing_path,'a')
    print("*"*100,end="\n"*3,file=f)
    print(alpha_name,end = "\n"*3,file=f)
    print(df,end="\n"*3,file = f)
    a = FT.get_top_return(alpha_name, negative= False, trade_type='long-only', transmission_rate=0.00025,
                            tax_rate=0.001, verbose=True,bt_price = "vwap",trade_period=6)
    print(a[1],end = "\n"*3+"*"*100 ,file = f)
    f.close()

Loading the configuration from /home/ShareFolder/lgc/Modules/Research/config/feature_bt_template


Loading Feature...:   2%|█▍                                                                                        | 92/5840 [00:00<00:08, 666.54it/s]

['AlphaNetV1_Shift_1_Sequence_20_10d_ret_standscalar_0', 'AlphaNetV1_Shift_1_Sequence_20_10d_ret_standscalar_5', 'AlphaNetV1_Shift_1_Sequence_20_10d_ret_standscalar_3', 'AlphaNetV1_Shift_1_Sequence_20_10d_ret_standscalar_4', 'AlphaNetV1_Shift_1_Sequence_20_10d_ret_standscalar_1', 'AlphaNetV1_Shift_1_Sequence_20_10d_ret_standscalar_8', 'AlphaNetV1_Shift_1_Sequence_20_10d_ret_standscalar_9', 'AlphaNetV1_Shift_1_Sequence_20_10d_ret_standscalar_7', 'AlphaNetV1_Shift_1_Sequence_20_10d_ret_standscalar_6', 'AlphaNetV1_Shift_1_Sequence_20_10d_ret_standscalar_2']


Loading Feature...:  59%|███████████████████████████████████████████████████▍                                   | 3449/5840 [00:03<00:01, 1232.53it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Calculating IC value...: 100%|█████████████████████████████████████████████████████████████████████████████████████| 584/584 [00:00<00:00, 639.90it/s]



IC table for feature "AlphaNetV1_Shift_1_Sequence_20_10d_ret_standscalar_0" (mean std IR):
+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|      period       |  IC_AlphaNetV1_1d  |  IC_AlphaNetV1_2d  |  IC_AlphaNetV1_3d  |  IC_AlphaNetV1_4d  |  IC_AlphaNetV1_5d  |
+===================+====================+====================+====================+====================+====================+
| 20190102-20190630 |   3.44 4.93 0.70   |   4.58 5.23 0.88   |   6.22 5.60 1.11   |   6.66 5.24 1.27   |   6.22 5.70 1.09   |
+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 20190701-20191231 |   2.02 4.17 0.48   |   3.24 3.24 1.00   |   4.04 4.14 0.98   |   3.86 4.38 0.88   |   3.63 4.22 0.86   |
+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 20200101-20200630

...


AlphaNetV1_Shift_1_Sequence_20_10d_ret_standscalar_6 back-test result for long-only mode:

Back test summary: "long-only" mode
+-------------------+-------------+----------+-----------+------------+----------+-------------+-------------------------+-------------------+------------+
|      period       |  stock_num  |  return  |  ret_std  |  ret_year  |  sharpe  |  win_ratio  |      max_dd|period      |  max_dd_day|date  |  turnover  |
+===================+=============+==========+===========+============+==========+=============+=========================+===================+============+
| 20190102-20190630 |     211     |   6.94   |   5.76    |   14.69    |   2.53   |    57.26    | 3.17(20190111-20190131) |  1.26(20190429)   |    8.49    |
+-------------------+-------------+----------+-----------+------------+----------+-------------+-------------------------+-------------------+------------+
| 20190701-20191231 |     215     |  -1.39   |   4.47    |   -2.64    |  -0.63   |    48.80 

Calculating IC value...: 100%|█████████████████████████████████████████████████████████████████████████████████████| 584/584 [00:00<00:00, 763.65it/s]



IC table for feature "AlphaNetV1_Shift_1_Sequence_20_10d_ret_standscalar_2" (mean std IR):
+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|      period       |  IC_AlphaNetV1_1d  |  IC_AlphaNetV1_2d  |  IC_AlphaNetV1_3d  |  IC_AlphaNetV1_4d  |  IC_AlphaNetV1_5d  |
+===================+====================+====================+====================+====================+====================+
| 20190104-20190630 |   3.29 5.53 0.60   |   4.10 5.58 0.73   |   4.42 5.16 0.86   |   5.33 5.96 0.90   |   5.51 6.30 0.87   |
+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 20190701-20191231 |   3.37 5.35 0.63   |   3.54 4.66 0.76   |   3.34 4.52 0.74   |   3.02 4.36 0.69   |   3.34 4.59 0.73   |
+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 20200101-20200630

Getting BT result for date: 20210531...: 100%|██████████████████████████████████████████████████████████████████████| 584/584 [00:06<00:00, 95.11it/s]



AlphaNetV1_Shift_1_Sequence_20_10d_ret_standscalar_2 back-test result for long-only mode:

Back test summary: "long-only" mode
+-------------------+-------------+----------+-----------+------------+----------+-------------+-------------------------+-------------------+------------+
|      period       |  stock_num  |  return  |  ret_std  |  ret_year  |  sharpe  |  win_ratio  |      max_dd|period      |  max_dd_day|date  |  turnover  |
+===================+=============+==========+===========+============+==========+=============+=========================+===================+============+
| 20190102-20190630 |     212     |   6.05   |   6.33    |   12.74    |   2.00   |    57.26    | 3.09(20190111-20190201) |  1.41(20190429)   |   14.52    |
+-------------------+-------------+----------+-----------+------------+----------+-------------+-------------------------+-------------------+------------+
| 20190701-20191231 |     214     |  -2.79   |   4.17    |   -5.27    |  -1.38   |    52.00 